В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from process_bank_churn import preprocess_data, preprocess_new_data

df = pd.read_csv('bank-customer-churn-prediction-dlu-course-c-2/train.csv')
# додала можливість не генерувати поліноміальні фічі порівняно з попереднім дз
datasets = preprocess_data(df, scaler_numeric = False, poly_numeric = False)

In [5]:
train_df = datasets['train_X']
val_df = datasets['val_X']
train_y = datasets['train_y']
val_y = datasets['val_y']
encoder = datasets['encoder']

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [7]:
knn = KNeighborsClassifier()
knn.fit(train_df, train_y)

KNeighborsClassifier()

In [9]:
def calculate_roc_auc(model, X, y):
    y_probs = model.predict_proba(X)[:, 1]
    fpr, tpr, thresholds = roc_curve(y, y_probs)
    roc_auc = auc(fpr, tpr)    
    return roc_auc

In [11]:
print(f"Area under ROC score on train dataset: {calculate_roc_auc(knn, train_df, train_y):.5f}")
print(f"Area under ROC score on validation dataset: {calculate_roc_auc(knn, val_df, val_y):.5f}")

Area under ROC score on train dataset: 0.81890
Area under ROC score on validation dataset: 0.58135


Модель погана, оскільки велика різниця roc між train та val датасетами - high variance - модель сильно підлаштувалась під навчальні дані та на тесті майже рандомно передбачує.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [13]:
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 30)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(train_df, train_y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [15]:
knn_best = knn_gs.best_estimator_

print(knn_gs.best_params_, knn_gs.best_score_)

{'n_neighbors': 28} 0.7945


In [17]:
print(f"Area under ROC score on train dataset: {calculate_roc_auc(knn_best, train_df, train_y):.5f}")
print(f"Area under ROC score on validation dataset: {calculate_roc_auc(knn_best, val_df, val_y):.5f}")

Area under ROC score on train dataset: 0.68331
Area under ROC score on validation dataset: 0.60629


In [19]:
model = DecisionTreeClassifier(random_state=42, max_depth = 10, max_leaf_nodes = 30)
model.fit(train_df, train_y)

print(f"Area under ROC score on train dataset: {calculate_roc_auc(model, train_df, train_y):.5f}")
print(f"Area under ROC score on validation dataset: {calculate_roc_auc(model, val_df, val_y):.5f}")

Area under ROC score on train dataset: 0.91930
Area under ROC score on validation dataset: 0.92812


Не можу сказати що модель стала краще порівняно з попередньою, без підбору гіперпараметра, оскільти на валідаційному наборі roc майже не змінилася (тепер 0.6 а було 0.58), перенавчання стало значно меншим але все ще є, та при цьому у моделі high bias - вона сильно недонавчена.

Дерево прийняття рішень дає значно кращий результат.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [24]:
dt = DecisionTreeClassifier(random_state=42)
params_dt = {'max_depth': np.arange(1, 21, 2),
             'max_leaf_nodes': np.arange(2, 11)}
dt_gs = GridSearchCV(dt, params_dt, cv=3, scoring='roc_auc')

In [26]:
%%time
dt_gs.fit(train_df, train_y)

CPU times: user 2.4 s, sys: 39.4 ms, total: 2.43 s
Wall time: 2.43 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='roc_auc')

In [28]:
dt_best = dt_gs.best_estimator_

print(dt_gs.best_params_, dt_gs.best_score_)

{'max_depth': 5, 'max_leaf_nodes': 9} 0.894190479642457


In [30]:
print(f"Area under ROC score on train dataset: {calculate_roc_auc(dt_best, train_df, train_y):.5f}")
print(f"Area under ROC score on validation dataset: {calculate_roc_auc(dt_best, val_df, val_y):.5f}")

Area under ROC score on train dataset: 0.89701
Area under ROC score on validation dataset: 0.91089


Ця модель трохи гірша ніж знайдена вручну оскільки треба підвищити параметр max_leaf_nodes.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [33]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 40),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [35]:
dt = DecisionTreeClassifier(random_state=42)
dt_rs = RandomizedSearchCV(
    dt,
    params_dt,
    n_iter = 40,
    cv=3,
    scoring="roc_auc",
    verbose=1,
    random_state = 28,
    refit=True
)

In [37]:
%%time
dt_rs.fit(train_df, train_y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
CPU times: user 726 ms, sys: 17.9 ms, total: 744 ms
Wall time: 741 ms


RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=40,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   random_state=28, scoring='roc_auc', verbose=1)

In [39]:
dt_random_search_best = dt_rs.best_estimator_

print(dt_rs.best_params_, dt_rs.best_score_)

{'splitter': 'best', 'min_samples_split': 10, 'min_samples_leaf': 8, 'max_leaf_nodes': 34, 'max_features': None, 'max_depth': 16, 'criterion': 'gini'} 0.9157347511867285


In [41]:
print(f"Area under ROC score on train dataset: {calculate_roc_auc(dt_random_search_best, train_df, train_y):.5f}")
print(f"Area under ROC score on validation dataset: {calculate_roc_auc(dt_random_search_best, val_df, val_y):.5f}")

Area under ROC score on train dataset: 0.92587
Area under ROC score on validation dataset: 0.92971


Так, ця модель краща за ту що знайшли за допомогою GridSearch. Глибина дерева 16 в цьому експерименті а була 5. 
max_leaf_nodes теж значно більша але я свідомо розширила діапазон.

На відміну від параметрів за замовчуванням, як ми залишили для GridSearch, у цьому випадку краще спрацювали параметри: 'min_samples_split': 10 а не 2, та 'min_samples_leaf': 8 а не 1. Інші залишилися без змін.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [113]:
test_raw_df = pd.read_csv('bank-customer-churn-prediction-dlu-course-c-2/test.csv')

In [124]:
test_df = preprocess_new_data(test_raw_df, encoder, poly = None, scaler = None, scaler_numeric = False, poly_numeric = False)['test_X']

In [127]:
def predict_test(model, inputs: pd.DataFrame):
    preds = model.predict(inputs)
    y_pred_proba = model.predict_proba(inputs)[:, 1]
    return y_pred_proba

In [ ]:
test_raw_df['Exited'] = predict_test(model, test_df)

In [ ]:
sample_submission = pd.read_csv('bank-customer-churn-prediction-dlu-course-c-2/sample_submission.csv')
sample_submission['Exited'] = test_raw_df['Exited']
sample_submission.head()
sample_submission.to_csv('submission_tree_rs.csv', index=False)

Для цієї моделі auroc = 0.9131, що гірше ніж для лог.регресії.